In [1]:
import pandas as pd
#Replace < > with appropriate file names/ folder paths

In [4]:
bamfile_metadata = pd.read_excel("<Path to TCGA data file with file IDs>")

In [5]:
fileids = bamfile_metadata['File ID']
sample_ids = {}
count = 0
for f in fileids:
    row = bamfile_metadata.loc[bamfile_metadata['File ID'] == f]
    sample_ids[f] = row['Sample ID'][count] 
    count+=1


In [2]:
import glob

all_files = glob.glob("<path to filtered SNP vcf files>")
indel_files = glob.glob("<path to filtered indel vcf files>")

all_files.extend(indel_files)


2520


In [3]:
with open("<path to *.list file containing all variants>", "w") as outfile:
    outfile.write("\n".join(all_files))

In [9]:
row_list = []

for file in all_files:
    with open(file) as f:
        for line in f:
            if not(line.startswith('#')):
                variant = line.strip().split("\t")
                chrom = variant[0]
                pos = variant[1]
                ref = variant[3]
                alt = variant[4]
                f = variant[-1].split(":")
                vcf_file = file.strip().split('/')
                file_id = vcf_file[2].split('_')
                sample_id = sample_ids[file_id[1]]
                freq = f[6]
                row = [chrom,pos,ref,alt,freq,sample_id]
                row_list.append(row)
            
var_df = pd.DataFrame(row_list)
var_df.columns = ['chr','pos','ref','alt','freq','sample_id']




In [11]:
#To store variant allele frequencies corresponding to each sample 
allele_freq = {}

for r in range(len(var_df)):
    row = var_df.iloc[r]
    key_list = [row['chr'],row['pos'],row['ref'],row['alt']]
    key = "-".join(key_list)
    if key not in allele_freq:
        allele_freq[key] = []
        allele_freq[key].append((row['sample_id'],row['freq']))
    else:
        allele_freq[key].append((row['sample_id'],row['freq']))



In [12]:
final_df = pd.DataFrame(columns = list(set(sample_ids.values())))
final_df.insert(0,'variant',allele_freq.keys())
for i,var in enumerate(allele_freq):
    for id_freq in allele_freq[var]:
        final_df.at[i, id_freq[0]] = id_freq[1]



In [13]:
final_df[['Chr','Pos','Ref','Alt']] = final_df.variant.str.split(pat='-',expand=True)
cols = final_df.columns.tolist()
new_cols = cols[-4:] + cols[1:len(cols)-4]
final_df = final_df[new_cols]
final_df = final_df.fillna('')

final_df.sort_values(by=['Pos'],inplace=True)
final_df = final_df.reset_index(drop=True)
final_df.to_excel("<path to variants excel file>")

In [ ]:
#Scripts to run to generate vcf file with unique variants

'''
Install picard tools and run the following command

java -jar picard.jar MergeVcfs \
          I=input_variant_files.list \
          O=output_variants.vcf.gz \
          D=chr11.dict

Install vcflib and run vcfstreamsort on output vcf file, followed by vcfuniq

Link to repository for details: https://github.com/vcflib/vcflib
'''

row_list = []
with open('<path to vcf file with unique variants>') as f:
    for line in f:
        if not(line.startswith('#')):
            variant = line.strip().split("\t")
            chrom = variant[0]
            pos = variant[1]
            ref = variant[3]
            alt = variant[4]
            row = [chrom,pos,ref,alt]
            row_list.append(row)
ann_df = pd.DataFrame(row_list)



In [16]:
'''
Run script to annotate vcf file using snpEff 

Link to snEff package: https://pcingola.github.io/SnpEff/

java -Xmx4g -jar snpEff/snpEff.jar hg38 ---vcf file name--- > ---annotated vcf file name---
'''
cf_data = pd.read_csv('<path to annotated vcf file>',sep='\t',comment='#',header=None)
vcf_data.columns = ['chr','pos','1','ref','alt','2','filter','info','extra','values']

In [18]:
info = vcf_data['info'].str.split(';',expand = True)

In [19]:
info_final = info[5].str.split('|',expand = True)

In [ ]:
to_add = info_final[[1,3,8,9,10]]
to_add.columns = ['sequence ontology','gene name','exon rank/number','HGVS.p','AA change']
to_add

In [23]:
var_data = pd.read_excel('<path to variants excel file>')

In [24]:
var_data_new = pd.concat([var_data, to_add], axis=1)

In [25]:
var_data_new.to_excel('<path to final variants excel file>')